In [2]:
import wmi
import gc
supported_cloudmusic_version = '2.10.7.4239'
wmic = wmi.WMI()
process = wmic.Win32_Process(name="cloudmusic.exe")
process = [p for p in process if '--type=' not in p.ole_object.CommandLine]
if not process:
    raise RuntimeError('No candidate process found')
elif len(process) != 1:
    raise RuntimeError('Multiple candidate processes found!')
else:
    process = process[0]
    from win32com.client import Dispatch
    ver_parser = Dispatch('Scripting.FileSystemObject')
    info = ver_parser.GetFileVersion(process.ExecutablePath)
    if info != supported_cloudmusic_version: raise RuntimeError(f'This version is not supported yet: {info}. Supported version: {supported_cloudmusic_version}')
    pid = process.ole_object.ProcessId
    del process
    del wmic
    del info
    del ver_parser
    del Dispatch
    gc.collect()

In [5]:
from pyMeow import open_process, get_module, r_float64, close_process, get_module, r_bytes, r_uint
import psutil
current_offset = 0xA65568
song_array_offset = 0xB16974

process = open_process(pid)
ps_util_process = psutil.Process(pid)
base_address = get_module(process, 'cloudmusic.dll')['base']
current = r_float64(process, base_address + current_offset)
songid_array = r_uint(process, base_address + song_array_offset)
song_id = r_bytes(process, songid_array, 0x14).decode('utf-16')
# close_process(process)
# gc.collect()

In [ ]:
import re
song_array_offset = 0xB15654
while True:
    try:
        songid_array = r_uint(process, base_address + song_array_offset)
        song_id = r_bytes(process, songid_array, 0x14).decode('utf-16')
        if song_id == '1458767558':
            print('found song array offset: ', hex(song_array_offset))
            break
        else:
            song_array_offset += 1
    except Exception as e:
        song_array_offset += 1

In [4]:
hex(base_address + 0xb15654)

'0x3ac5654'

In [5]:
import ctypes
from ctypes import wintypes

user32 = ctypes.windll.user32

WNDENUMPROC = ctypes.WINFUNCTYPE(
    wintypes.BOOL,
    wintypes.HWND,    # _In_ hWnd
    wintypes.LPARAM,) # _In_ lParam

user32.EnumWindows.argtypes = (
   WNDENUMPROC,      # _In_ lpEnumFunc
   wintypes.LPARAM,) # _In_ lParam

user32.IsWindowVisible.argtypes = (
    wintypes.HWND,) # _In_ hWnd

user32.GetWindowThreadProcessId.restype = wintypes.DWORD
user32.GetWindowThreadProcessId.argtypes = (
  wintypes.HWND,     # _In_      hWnd
  wintypes.LPDWORD,) # _Out_opt_ lpdwProcessId

user32.GetWindowTextLengthW.argtypes = (
   wintypes.HWND,) # _In_ hWnd

user32.GetWindowTextW.argtypes = (
    wintypes.HWND,   # _In_  hWnd
    wintypes.LPWSTR, # _Out_ lpString
    ctypes.c_int,)   # _In_  nMaxCount

def get_title(pid) -> str:
    ret = ''
    @WNDENUMPROC
    def enum_proc(hWnd, lParam):
        nonlocal ret
        if user32.IsWindowVisible(hWnd):
            _pid = wintypes.DWORD()
            user32.GetWindowThreadProcessId(hWnd, ctypes.byref(_pid))
            length = user32.GetWindowTextLengthW(hWnd) + 1
            title = ctypes.create_unicode_buffer(length)
            user32.GetWindowTextW(hWnd, title, length)
            if _pid.value == pid: ret = title.value
        return True
    user32.EnumWindows(enum_proc, 0)
    return ret

get_title(pid)

'星辰大海 - 黄霄雲'